# 1 Introduction

General overview over the database system.

### Postgresql

[Postgresql](https://www.postgresql.org/) is an open source database system. At the moment, KLUSTA output and BNT scores are saved into it and can be retrieved by connecting to it via [Pandas](http://pandas.pydata.org/pandas-docs/stable/index.html), which creates a convenient interface with the postgres structure. 

Postgres is organized in databases and tables. At the moment, all recording data / scores are stored under the database *data_1*. Data_1 has many tables which are partially linked to each other and ordered in a hierarchy. This eliminates the possiblity of double entries or entries at the bottom of the hierarchy, which have no relation to the dataset as a whole. For example, there is no way that a gridscore can exist somewhere, if not a fitting tetrode and session have been found. On the other hand, deleting a tetrode for example, will delete all children of this tetrode, meaning all clusters and their linked scores. This keeps everything nicely in sync. Here is an example of a hierarchy of tables (from top to bottom): 

- **META** contains the name of the experimenter, animal ID, session timestamp (the AXONA timestamp for the base session of every data folder)  
- **TETRODES** contains tetrode number, kwik filename, name of the experimenter, animal ID, session timestamp and *inherits* the last three from *META*. This means that upon updating of an entry in *META*, all the linked entries in *TETRODES* will also be updated
- **CLUSTERS** contains cluster_no, and everything from *TETRODES*, which it inherits from that table the same way as *TETRODES* inherits from *META*, meaning again that there can't be any clusters, which don't relate to tetrodes, which don't relate to meta entries. 
- **RATEMAPS** contains the ratemaps, additional parameters that were used for the calculation, and inherits everything from *CLUSTERS* ... 
- ... and so on


Similarly, there are tables that branch off the hierarchy described above: The **SESSIONS** table is another organizing table which inherits from *META*, but then has children on its own which are animal tracking and LFP data, since this data are organized in sessions and not Tetrodes / Clusters. 


In addition to the hierarchical structure, single tables have **uniqe** constraints, meaning that a single entry in a specific table cannot exist twice. These unique constraints can span multiple column names. For example, inside the table **TETRODES**, the columns for tetrode number, experimenter name, animal ID and session timestamp together must be unique for every entry in that table. Another entry that matches these parameters and is attempted to be written to the database will be refused.

In practice it works like that: Whenever an `analysis_pipeline.py` job is run at the moment, the spatial analysis routine (which comes first) takes care of cleaning exisiting entries and creating empty ones that are then filled with for example ratemaps, gridscores. The subsequent stimulus analysis does not have the right to purge the existing data, but will write into the structure. That makes sure that if you change clusters in klustakwik that have already been analyzed and run `analysis_pipeline.py` a second time, there are no "old" clusters or scores lying around. 

### Organisation of database ***data_1***

By default a connection is made to the database *data_1*. A complete table structure and entries can be found below. BNT tables run in parallel to the KLUSTA tables since right now these two don't completely overlap: There are more BNT entries than there are KLUSTA entries and the organizing principle in KLUSTA is tetrodes (x.kwik), while it is single cells for the output of BNT. 

#### KLUSTA

You can find the name of each table (**_tb**) and a description of its field names on the right. Unique entries (or rather the combination of entries that together have to be unique) are shown in bold. No inheritance (hierarchy) is shown, but basically all data that requires single cell level entries (for example ratemaps_tb) inherits from **clusters_tb** which in turn inherits from all tables above it (tetrodes_tb,meta_tb). Similarly, LFP and tracking data inherit from **sessions_tb** which inherits from meta_tb again. This is a useful reference for the retrieval of data (see next notebooks). Click here to open the table structure / description below in google docs: [Table definitions on google docs](https://docs.google.com/spreadsheets/d/117LVxOBbHKpyERKeX4GJqCP1C1JOLP4fVy367vUMRrU/edit?usp=sharing)

In [1]:
%%html
<iframe src="https://docs.google.com/spreadsheets/d/117LVxOBbHKpyERKeX4GJqCP1C1JOLP4fVy367vUMRrU/pubhtml?gid=677642759&amp;single=true&amp;widget=true&amp;headers=false" height="900" width="100%"></iframe>

##### BNT 

[Table definitions BNT on google docs](https://docs.google.com/spreadsheets/d/1spaNEHaK1aaebwQpoSLRfDrDwdBvcVWw_P-1IMbsgJE/edit?usp=sharing)

In [2]:
%%html
<iframe src="https://docs.google.com/spreadsheets/d/1spaNEHaK1aaebwQpoSLRfDrDwdBvcVWw_P-1IMbsgJE/pubhtml?gid=677642759&amp;single=true&amp;widget=true&amp;headers=false" height="900" width="100%"></iframe>